In [3]:
import numpy as np
import random
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

In [4]:
alice_angles = [0, np.pi/4, np.pi/2]
bob_angles = [np.pi/4, np.pi/2, 3*np.pi/4]
simulator = AerSimulator()

In [5]:
def create_bell_pair():
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    return qc

In [6]:
def measure_in_basis(qc, qubit, angle):
    qc.ry(-2 * angle, qubit)
    qc.measure(qubit, qubit)

In [7]:
def e91_simulation(n):
    data = []
    for _ in range(n):
        qc = create_bell_pair()

        a_idx = random.randint(0, 2)
        b_idx = random.randint(0, 2)
        a_angle = alice_angles[a_idx]
        b_angle = bob_angles[b_idx]

        measure_in_basis(qc, 0, a_angle)
        measure_in_basis(qc, 1, b_angle)

        tqc = transpile(qc, simulator)
        result = simulator.run(tqc, shots=1).result()
        counts = result.get_counts()
        outcome = list(counts.keys())[0]

        data.append({
            "alice_angle_idx": a_idx,
            "bob_angle_idx": b_idx,
            "outcome": outcome
        })
    return data

In [8]:
def calculate_chsh(data):
    counts = {(0, 0): [], (0, 1): [], (1, 0): [], (1, 1): []}
    for d in data:
        a, b = d["alice_angle_idx"], d["bob_angle_idx"]
        if (a, b) in counts:
            outcome = d["outcome"]
            counts[(a, b)].append(1 if outcome in ['00', '11'] else -1)
    def mean(l): return sum(l)/len(l) if l else 0
    E = {key: mean(vals) for key, vals in counts.items()}
    S = abs(E[(0, 0)] - E[(0, 1)] + E[(1, 0)] + E[(1, 1)])
    return S

In [9]:
def calculate_qber(data):
    mismatch = total = matched_bits = 0
    for d in data:
        if d["outcome"] in ['00', '11']:
            matched_bits += 1
        total += 1
        if d["outcome"] not in ['00', '11']:
            mismatch += 1
    qber = mismatch / total if total else 0
    return qber, matched_bits

In [10]:
def calculate_key_rate(qber, total_rounds, matched_bits):
    R0 = matched_bits / total_rounds
    return R0 * (1 - 2 * qber)

In [11]:
def final_key(data):
    secret_key = ''
    for d in data:
        if d["outcome"] == '00':
            secret_key += '0'
        elif d["outcome"] == '11':
            secret_key += '1'
    return secret_key

In [12]:
try:
    import matplotlib
    import matplotlib.pyplot as plt
    interactive = matplotlib.get_backend().lower() not in ['agg', 'template']
except Exception:
    plt = None
    interactive = False

def plot_chsh(data):
    if plt is None:
        raise ModuleNotFoundError("matplotlib is not installed.")
    S = calculate_chsh(data)
    plt.axhline(y=2, color='r', linestyle='--', label='Classical Limit')
    plt.bar(["CHSH"], [S], color='green')
    plt.title("CHSH Violation")
    plt.ylim(0, 4)
    plt.legend()
    if interactive:
        plt.show()
    else:
        print("⚠️ Matplotlib backend is non-interactive.")

def plot_qber(qber):
    if plt is None:
        raise ModuleNotFoundError("matplotlib is not installed.")
    plt.bar(["QBER"], [qber * 100], color='orange')
    plt.title("Quantum Bit Error Rate (%)")
    plt.ylim(0, 100)
    if interactive:
        plt.show()
    else:
        print("⚠️ Plot not shown.")

def plot_key_rate(rate):
    if plt is None:
        raise ModuleNotFoundError("matplotlib is not installed.")
    plt.bar(["Key Rate"], [rate], color='blue')
    plt.title("Final Usable Key Rate")
    plt.ylim(0, 1)
    if interactive:
        plt.show()
    else:
        print("⚠️ Plot not shown.")

In [ ]:
rounds = int(input("Enter number of rounds (default 1000): ") or 1000)
results = e91_simulation(rounds)
qber, matched_bits = calculate_qber(results)
chsh = calculate_chsh(results)
key_rate = calculate_key_rate(qber, rounds, matched_bits)
secret_key = final_key(results)

print(f"Secret key: {secret_key}")
print(f"CHSH ( 2 < S >= 2.828): {chsh:.3f}")
print(f"QBER ( QBER < 11% ): {qber*100:.2f}%")
print(f"Key Rate: {key_rate:.4f}")

plot_chsh(results)
plot_qber(qber)
plot_key_rate(key_rate)

Enter number of rounds (default 1000):  100
